In [18]:
from __future__ import print_function
from elasticsearch import Elasticsearch
import sys
from operator import add
from pyspark import SparkContext

import numpy as np
import csv
import math

#period: the days of period you choose
#✅WELL DONE
def get_csv_data(filename):
    filename = "file:/Users/nancywu/sparkhadoop/datatest/"+filename+".csv"
    File = sc.textFile(filename)
    File.map(lambda line: line.split(","))
    File.filter(lambda line: len(line) > 0)
    File.map(lambda line: (line[0], line[1]))
    #print(File.first())
    data = File.collect()
    stock_text = [d.split(",") for d in data]
    #print(stock_text[1][1])
    start, period = 1, 90
    predict_period = 2
    open_price = [float(stock_text[i][1]) for i in range(start, start + period)]
    close_price = [float(stock_text[i][4]) for i in range(start, start + period)]
    S0 = float(stock_text[91][-1])
    #print (open_price)
    True_price = ["True_price"]
    [True_price.append(float(stock_text[j][-1])) for j in range(1, period + predict_period)[::-1]]
    #print(True_price)
    print ("get_csv_data: done")
    #print ( open_price,close_price,S0,True_price)
    return open_price,close_price,S0,True_price

#mu: the mean of sample training size
#v:
#NUMPY WRONG!!solved it already on Nov.28

def simulation(filename):
    open_price,close_price,S0,True_price = get_csv_data(filename)
    #print ("mu")
    mu= (np.array(open_price).mean()+np.array(close_price).mean())*0.5
    #print(mu)
    t = 1.0/365
    #v = ((np.array(open_price).var() + np.array(close_price).var())/(np.array(open_price).mean()+np.array(close_price).mean())) * 0.5
    #print ("various")
    open_close_average_list = list(map(lambda x: float(x[0]+x[1])*0.5, zip(open_price, close_price)))
    v= np.array(open_close_average_list).var()
    #print (v)
    n,M = 90,1000
    result = ["Prediction"]
    for i in range(n+1):
        prediction = sc.parallelize(np.random.normal(0,1,M)).map(lambda x: (mu-0.5*v)*t*i+v*x*(t*i)**0.5).map(lambda x: math.exp(x)).map(lambda x: S0*x)
        result.append(prediction.mean())
    print ("==========predictresult pass========")
    return True_price, result

def generation_output(True_price,prediction):
    Date = ["Date", u'2015-11-19', u'2015-11-20', u'2015-11-23', u'2015-11-24', u'2015-11-25', u'2015-11-27',
                       u'2015-11-30', u'2015-12-01', u'2015-12-02', u'2015-12-03', u'2015-12-04', u'2015-12-07',
                       u'2015-12-08', u'2015-12-09', u'2015-12-10', u'2015-12-11', u'2015-12-14', u'2015-12-15',
                       u'2015-12-16', u'2015-12-17', u'2015-12-18', u'2015-12-21', u'2015-12-22', u'2015-12-23',
                       u'2015-12-24', u'2015-12-28', u'2015-12-29', u'2015-12-30', u'2015-12-31', u'2016-01-04',
                       u'2016-01-05', u'2016-01-06', u'2016-01-07', u'2016-01-08', u'2016-01-11', u'2016-01-12',
                       u'2016-01-13', u'2016-01-14', u'2016-01-15', u'2016-01-19', u'2016-01-20', u'2016-01-21',
                       u'2016-01-22', u'2016-01-25', u'2016-01-26', u'2016-01-27', u'2016-01-28', u'2016-01-29',
                       u'2016-02-01', u'2016-02-02', u'2016-02-03', u'2016-02-04', u'2016-02-05', u'2016-02-08',
                       u'2016-02-09', u'2016-02-10', u'2016-02-11', u'2016-02-12', u'2016-02-16', u'2016-02-17',
                       u'2016-02-18', u'2016-02-19', u'2016-02-22', u'2016-02-23', u'2016-02-24', u'2016-02-25',
                       u'2016-02-26', u'2016-02-29', u'2016-03-01', u'2016-03-02', u'2016-03-03', u'2016-03-04',
                       u'2016-03-07', u'2016-03-08', u'2016-03-09', u'2016-03-10', u'2016-03-11', u'2016-03-14',
                       u'2016-03-15', u'2016-03-16', u'2016-03-17', u'2016-03-18', u'2016-03-21', u'2016-03-22',
                       u'2016-03-23', u'2016-03-24', u'2016-03-28', u'2016-03-29', u'2016-03-30', u'2016-03-31',
                       u'2016-04-01']
    output = []
    for i in range(len(Date)):
        tmp = [Date[i],prediction[i],True_price[i]]
        output.append(tmp)
    return output
#Do not  test it yet⚠️
#def writeToElastic(fileindex,es,filename,stock_text):
    index_list = stock_text[0]
    for i in stock_text[1:]:
        d = {}
        for x, y in zip(index_list, i):
            if x == "Date":
                Date = y
                d[x] = y
            else:
                d[x] = float(y)
        es.index(index=fileindex, doc_type=filename, id=Date, body=d)
        
if __name__ =="__main__":
    #sc = SparkContext(appName="Monte Carlo")
    Ticker = sc.textFile("file:/Users/nancywu/sparkhadoop/datatest/Tickertest.csv")
    filelist = Ticker.map(lambda f: f.split(",")).collect()
    #l = Ticker.collect()
    #filelist = l[0].split(",")
    print(filelist)
    es = Elasticsearch()
    print("===========start============")
    for f in filelist:
        try:
            name = f[0]+".csv"
            print (name,"simulation",f[0])
            True_price,prediction = simulation(f[0])
            print("=====simulation pass=====")
            print(len(True_price),len(prediction))
            output = generation_output(True_price,prediction)
            print("=======output pass=======")
            print (output)
            print (name)
            sc.parallelize(output).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + name)
            #writeToElastic("predictvalue",es,name,output)
        except:
            print("No service for this stock")




[[u'AA']]
===========start============
AA.csv simulation AA
get_csv_data: done
==========predictresult pass========
=====simulation pass=====
92 92
=======output pass=======
[['Date', 'Prediction', 'True_price'], [u'2015-11-19', 8.722296, 8.722296], [u'2015-11-20', 8.917456362755491, 8.652596], [u'2015-11-23', 9.11962073642547, 9.030961], [u'2015-11-24', 9.35379450286787, 9.050875], [u'2015-11-25', 9.590326143868722, 9.369498], [u'2015-11-27', 9.85309749543809, 9.279885], [u'2015-11-30', 10.12728409459718, 9.319713], [u'2015-12-01', 10.250367496307865, 9.439196], [u'2015-12-02', 10.547840367422165, 9.309756], [u'2015-12-03', 10.807498135923503, 8.951305], [u'2015-12-04', 11.120064182933135, 9.299799], [u'2015-12-07', 11.408000876000822, 9.00109], [u'2015-12-08', 11.671375652332191, 8.483329], [u'2015-12-09', 11.997060708482131, 8.702381], [u'2015-12-10', 12.337257306217571, 9.130531], [u'2015-12-11', 12.616782944511531, 8.821865], [u'2015-12-14', 12.819194865447217, 8.961262], [u'2015-